# Checking Job Sets for Consistency and Quality

***

# Import Modules

In [1]:
%%capture
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.insert(
    0,
    os.path.join(
        os.environ["PROJ_irox"],
        "workflow",
        ),
    )

sys.path.insert(
    0,
    os.path.join(
        os.environ["PROJ_irox"],
        "data",
        ),
    )

from an_data_processing import load_df

from IPython.display import display

###########################################################

from proj_data_irox import (
    data_dir,
    )

# Python Modules
import subprocess
import numpy as np

import pandas as pd
# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from ase.visualize import view

from ase.io import Trajectory

# Script Inputs

In [2]:
prop_name_list = [
    'bulk_system',
#     'coverage',
    'coverage_type',
    'facet',
    ]

# Read and Process Data Frame

In [3]:
df_pourbaix, df_ads, df_surf = load_df(
    from_file=True,
    root_dir=data_dir,
    data_dir=data_dir,
    file_name="df_master.pickle",
    process_df=True,
    )

df_m = df_ads
df_m.loc[df_m["coverage_type"] == "O-4_OH-0", "coverage_type"] = "o_covered"
df_m.loc[df_m["coverage_type"] == "O-2_OH-0", "coverage_type"] = "o_covered_2"
df_m.loc[df_m["coverage_type"] == "O-2_OH-2", "coverage_type"] = "h_covered"

Attempting to load df from pickle


In [4]:
def create_energetics_table(
    atoms_dir,
    sys_name_i,
    root_dir,
    ):
    """
    """
    #| - create_energetics_table
    os.chdir(
        os.path.join(
            atoms_dir,
            sys_name_i,
            ),
        )

    bash_comm = os.path.join(
        os.environ["PROJ_irox"],
        "scripts",
        "get_energies_top_sites.py",
        )

    result = subprocess.run(
        [bash_comm],
        stdout=subprocess.PIPE,
        )

    out_i = result.stdout.decode("utf-8")

    with open("energetics_summary.txt", "w") as text_file:
        text_file.write(out_i)
#     out_i = [i for i in out_i.split("\n") if i != '']

    os.chdir(root_dir)
    #__|

# Creating Traj Objects for Each 'Job Set'

In [5]:
import datetime
d = datetime.datetime.today()
date_i = str(d.year) + str(d.month) + str(d.day)
print(date_i)

20181222


In [6]:
from ase import io

root_dir = os.getcwd()

# atoms_dir = "00_job_sets_atoms_181116_02"

num_atoms_dirs = len([i for i in os.listdir(".") if "job_sets_atoms" in i])
# atoms_dir = "00_job_sets_atoms/" + str(num_atoms_dirs).zfill(2) + "_job_sets_atoms"
atoms_dir = "00_job_sets_atoms_" + date_i + "/" + str(num_atoms_dirs).zfill(2) + "_job_sets_atoms"

if not os.path.exists(atoms_dir):
    os.makedirs(atoms_dir)

groups = df_m.groupby(
    by=prop_name_list,
    )
for i_ind, (props_i, group_i) in enumerate(groups):
#     images_list = [i for i in group_i["atoms_object"].tolist() if i is not None]
#     final_atoms_list = [i[-1] for i in images_list]

    sys_name_i = str(i_ind).zfill(2) + "_" + "___".join(props_i)

    sys_i_dir = os.path.join(
        atoms_dir,
        sys_name_i,
        )

    if not os.path.exists(sys_i_dir):
        os.makedirs(sys_i_dir)

    for j_cnt, row_j in group_i.iterrows():

        name_j = row_j["adsorbate"]

        if not pd.isna(row_j["ooh_direction"]):
            name_j += "_" + row_j["ooh_direction"]

#         name_j += ".traj"
        name_j += ".json"

        if row_j["atoms_object"] is None:
            continue
        else:
            atoms_j = row_j["atoms_object"][-1]

            atoms_j_full_path = os.path.join(
                atoms_dir,
                sys_name_i,
                name_j,
                )

            print(atoms_j_full_path)

            io.write(
                atoms_j_full_path,
                atoms_j,
                )

    create_energetics_table(
        atoms_dir,
        sys_name_i,
        root_dir,
        )

00_job_sets_atoms_20181222/02_job_sets_atoms/00_IrO2___h_covered___100/bare.json
00_job_sets_atoms_20181222/02_job_sets_atoms/00_IrO2___h_covered___100/ooh_up.json
00_job_sets_atoms_20181222/02_job_sets_atoms/00_IrO2___h_covered___100/ooh_down.json
00_job_sets_atoms_20181222/02_job_sets_atoms/00_IrO2___h_covered___100/ooh_deprotonated.json
00_job_sets_atoms_20181222/02_job_sets_atoms/00_IrO2___h_covered___100/o.json
00_job_sets_atoms_20181222/02_job_sets_atoms/00_IrO2___h_covered___100/oh.json
00_job_sets_atoms_20181222/02_job_sets_atoms/01_IrO2___h_covered___110/bare.json
00_job_sets_atoms_20181222/02_job_sets_atoms/01_IrO2___h_covered___110/ooh_up.json
00_job_sets_atoms_20181222/02_job_sets_atoms/01_IrO2___h_covered___110/ooh_down.json
00_job_sets_atoms_20181222/02_job_sets_atoms/01_IrO2___h_covered___110/ooh_down.json
00_job_sets_atoms_20181222/02_job_sets_atoms/01_IrO2___h_covered___110/ooh_deprotonated.json
00_job_sets_atoms_20181222/02_job_sets_atoms/01_IrO2___h_covered___110/ooh

In [7]:
import datetime
d = datetime.datetime.today()
date_i = str(d.year) + str(d.month) + str(d.day)
print(date_i)

20181222


In [8]:
# Comparing Magmoms

# display(df_m[
#     (df_m["bulk_system"] == "IrO2") & \
#     (df_m["coverage_type"] == "o_covered") & \
#     (df_m["facet"] == "100")
# #     (df_m[] == "") & \
#     ])


# # row_slab = df_m.loc[53]
# row_slab = df_m.loc[36]
# slab = row_slab.atoms_object[-1]

# # row_ads = df_m.loc[63]
# row_ads = df_m.loc[51]
# ads = row_ads.atoms_object[-1]

# from ase_modules.get_G import Get_G

# GG = Get_G(
#     slab,
#     ads,
#     default_vib_bool=True,
#     get_E=False,
#     index=-1,
#     quiet=False,
#     )

# GG.compare_magmoms()
# # GG.slab

# # view(slab)
# # view(ads)

# for i_cnt, row_i in df_m.iterrows():   
#     atoms_i = row_i["atoms_object"]
#     if atoms_i is None:
#         continue
    
#     for image_i in atoms_i:
#         magmoms_i = image_i.get_magnetic_moments()
#         image_i.set_initial_magnetic_moments(magmoms_i)

#             traj_i = Trajectory(
#                 atoms_j_full_path,
#                 "w",
#                 )

#             traj_i.write(
#                 atoms_j,
#                 energy=1.2345,
#                 )


# df_m[
#     (df_m["bulk_system"] == "IrO2") & \
#     (df_m["coverage_type"] == "o_covered") & \
#     (df_m["facet"] == "100")
#     ]

# df_m[
#     (df_m["bulk_system"] == "IrO2") & \
#     (df_m["coverage_type"] == "h_covered") & \
#     (df_m["facet"] == "100")
#     ].path.tolist()

# # ads_magmoms = ads.get_magnetic_moments()
# # ads.set_initial_magnetic_moments(ads_magmoms)

# for image_i in row_ads["atoms_object"]:
# #     row_ads["atoms_object"][0].get_magnetic_moments()
    
#     try:
#         magmoms_i = image_i.get_magnetic_moments()
#         print(magmoms_i[0])
#     except:
#         print("**")


# import numpy as np
# import pandas as pd
# import plotly.plotly as py
# import plotly.graph_objs as go
# import colorlover as cl
# from IPython.display import HTML

# My Modules
# from orr_reaction.orr_fed_plot import ORR_Free_E_Plot
# from orr_reaction.orr_fed_plot import Scaling_Relations_Plot

# Project Data
# from proj_data_irox import (
#     surface_energies,
#     smart_format_dict_volcano,
#     color_palettes,
#     system_color_map,
#     max_surf_e,
#     min_surf_e,
#     proj_dir_name,
#     smart_format_dict,
#     )

# Script Inputs

# save_plot = False
# prop_name_list = [
#     'bulk_system',
# #     'coverage',
#     'coverage_type',
#     'facet',
#     ]